In [18]:
import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.models.DirectRanker import directRanker
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [2]:

file_path = './data/adult.csv'
data = pd.read_csv(file_path)
data.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [20]:
label_encoder_x = LabelEncoder()
label_encoder_y = LabelEncoder()
X_data = data.loc[:, data.columns != ' <=50K']
y_data = data[' <=50K']
string_data = [' State-gov', ' Bachelors', ' Never-married', ' Adm-clerical', ' Not-in-family',
               ' White', ' Male', ' United-States']
X_data[string_data] = X_data[string_data].apply(label_encoder_x.fit_transform)
y_data = label_encoder_y.fit_transform(y_data)

/var/folders/nx/wz1vqyld77ncdjz679wgbm_m0000gn/T/ipykernel_5012/809402148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_data[string_data] = X_data[string_data].apply(label_encoder_x.fit_transform)


In [4]:
full_df = pd.concat([X_data, pd.DataFrame(y_data, columns=['y'])], axis=1)
full_df.head(5)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,y
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,0


In [5]:
X_data_0 = full_df[full_df['y'] == 0]
X_data_1 = full_df[full_df['y'] == 1]

In [6]:
len(X_data_0), len(X_data_1)

(24719, 7841)

In [7]:
X_data_0 = X_data_0[:len(X_data_1)]
X_data_0 = X_data_0.loc[:, X_data_0.columns != 'y']
X_data_1 = X_data_1.loc[:, X_data_1.columns != 'y']

In [8]:
y = np.ones(len(X_data_1))

In [9]:
X_data_0.head(5)

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
0,50,6,83311,9,13,2,4,0,4,1,0,0,13,39
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39
3,28,4,338409,9,13,2,10,5,2,0,0,0,40,5
4,37,4,284582,12,14,2,4,5,4,0,0,0,40,39


In [10]:
X_tensor_0 = torch.tensor(X_data_0.values, dtype=torch.float32)
X_tensor_1 = torch.tensor(X_data_1.values, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [11]:
X_tensor_0.shape, X_tensor_1.shape, y_tensor.shape

(torch.Size([7841, 14]), torch.Size([7841, 14]), torch.Size([7841]))

In [12]:
y_tensor.dtype

torch.float32

In [13]:
(torch.unsqueeze(y_tensor, 1)).shape

torch.Size([7841, 1])

In [12]:
labels = torch.randint(0, 2, (y_tensor.size(0), ), dtype=torch.float32)
test_x0 = X_tensor_0.clone()
test_x1 = X_tensor_1.clone()
for i, label in enumerate(labels):
    if label == 0:
        test_x0[i], test_x1[i] = test_x1[i], test_x0[i]

In [16]:
model = directRanker(num_features=14, hidden_layers=[10, 5])
n_epochs = 1000
#print(list(model.parameters()))
lr = 0.1
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
# training
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(test_x0, test_x1)
    y_pred = torch.squeeze(y_pred)
    loss = loss_fn(labels, y_pred)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:    
        print(loss)
        for name, param in model.named_parameters():
            if param.grad is None:
                print(name)
            else:
                print(param.grad.data.sum())
                break


tensor(0.9945, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.)
tensor(0.9758, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.)
tensor(0.9758, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.)
tensor(0.9758, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.)
tensor(0.9758, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.)
tensor(0.9758, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.)


KeyboardInterrupt: 

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

215

In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-9.6254e-01, -7.7350e-01, -1.2496e+00,  2.3286e+00,  1.1255e+00,
           7.7505e-01,  3.6949e-01, -6.2064e-01, -6.3214e-01, -6.8152e-01,
           5.9237e-01,  8.9321e-01, -2.4131e+00,  5.9124e-01],
         [-4.4335e-01,  1.4571e+00,  2.9654e-01,  5.6087e-01,  1.7136e+00,
           1.1585e-01,  2.0781e-01,  3.4840e-01,  4.0644e-01, -7.1585e-01,
          -9.1495e-01,  6.2616e-01, -7.6774e-01,  9.3473e-01],
         [ 2.2000e+00,  1.1954e+00,  4.3366e-01, -9.4003e-01,  9.1544e-01,
           4.7165e-01,  3.0606e-01, -1.1699e-01, -3.9122e-01,  1.7600e-01,
           2.3944e-01,  1.6623e+00, -7.5292e-02, -1.1068e+00],
         [-1.3098e-01,  3.9877e-01, -1.2702e+00,  1.3591e+00,  1.3170e+00,
           1.3941e+00, -7.3637e-01, -1.2095e+00,  2.1095e+00, -6.4229e-01,
          -2.3668e-01,  1.0204e+00, -2.9939e-01,  1.6530e+00],
         [-2.2035e+00, -1.8394e+00, -1.1027e+00,  2.3633e+00, -1.2846e+00,
          -7.7130e-01,  1.6945e+00,  1.0791e+00,  

In [ ]:
for name, param in model.named_parameters():
  print(name, param.requires_grad)

layers.0.weight True
layers.0.bias True
layers.1.weight True
layers.1.bias True
ranking_layer.weight True
ranking_layer_cls.weight True


In [ ]:
X_tensor_0[0].shape

torch.Size([14])

In [ ]:
model.eval()
y_pred = model.forward(X_tensor_0[4], X_tensor_1[9])
y_pred

tensor([0.], grad_fn=<TanhBackward0>)

In [ ]:
model.eval()
y_pred = model.forward(X_tensor_1[2], X_tensor_0[0])
y_pred

tensor([0.], grad_fn=<TanhBackward0>)

In [ ]:
def generate_direct_ranker_dataset(num_samples, num_features):
    # Erzeugen von zwei Tensoren mit zufälligen Daten
    tensor_A = torch.randn(num_samples, num_features, dtype=torch.float32)
    tensor_B = torch.randn(num_samples, num_features, dtype=torch.float32)

    # Erzeugen von zufälligen Labels (0 oder 1)
    labels = torch.randint(0, 2, (num_samples,), dtype=torch.float32)

    return tensor_A, tensor_B, labels

# Beispiel: Erzeugen eines Datensatzes mit 100 Samples, jeder mit 10 Features
num_samples = 100
num_features = 10
tensor_A, tensor_B, labels = generate_direct_ranker_dataset(num_samples, num_features)

print(tensor_A.shape, tensor_B.shape, labels.shape)


torch.Size([100, 10]) torch.Size([100, 10]) torch.Size([100])


In [ ]:
model = directRanker(num_features=10)
n_epochs = 1000
#print(list(model.parameters()))
lr = 0.1
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
# training
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(tensor_A, tensor_B)
    y_pred = torch.squeeze(y_pred)
    loss = loss_fn(labels, y_pred)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:    
        print(loss)

tensor(0.8468, grad_fn=<MseLossBackward0>)
tensor(0.0015, grad_fn=<MseLossBackward0>)
tensor(0.0008, grad_fn=<MseLossBackward0>)
tensor(0.0008, grad_fn=<MseLossBackward0>)
tensor(0.0002, grad_fn=<MseLossBackward0>)
tensor(0.0003, grad_fn=<MseLossBackward0>)
tensor(0.0002, grad_fn=<MseLossBackward0>)
tensor(0.0002, grad_fn=<MseLossBackward0>)
tensor(0.0007, grad_fn=<MseLossBackward0>)
tensor(0.0005, grad_fn=<MseLossBackward0>)
